In [2]:
import os
import torch 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import argparse
from sam2.build_sam import build_sam2_video_predictor

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("Using CPU, not recommended.")

# For efficiency
if device.type == "cuda":
    torch.autocast("cuda", dtype = torch.bfloat16).__enter__()

# Change this to the directory where you cloned this repo
BASE_PATH = "C:\\Users\\syeda\\OneDrive\\Desktop\\4th Year\\COSC419\\sam2-tests"
CHKPT_PATH = os.path.join(BASE_PATH, "checkpoints", "sam2.1_hiera_large.pt")
CONFIG_PATH = os.path.join(BASE_PATH, "sam2", "configs", "sam2.1", "sam2.1_hiera_l.yaml")
FRAMES_PATH = os.path.join(BASE_PATH, "sample_frames")

In [ ]:
torch.cuda.empty_cache()  # remove lingering allocations
model_allocated = torch.cuda.memory_allocated(device=device)
model_reserved = torch.cuda.memory_reserved(device=device)

print(f"Memory before loading model:")
print(f"  Allocated: {model_allocated / (1024 ** 3):.2f} GB")
print(f"  Reserved: {model_reserved / (1024 ** 3):.2f} GB")

Memory before loading model:
  Allocated: 1.87 GB
  Reserved: 2.15 GB


In [ ]:
predictor = build_sam2_video_predictor(CONFIG_PATH, CHKPT_PATH, device=device)

model_allocated = torch.cuda.memory_allocated(device=device)
model_reserved = torch.cuda.memory_reserved(device=device)

print(f"Memory after loading model:")
print(f"  Allocated: {model_allocated / (1024 ** 3):.2f} GB")
print(f"  Reserved: {model_reserved / (1024 ** 3):.2f} GB")

Memory after loading model:
  Allocated: 1.87 GB
  Reserved: 2.92 GB


In [11]:
inference_state = predictor.init_state(video_path=FRAMES_PATH, offload_video_to_cpu=True, async_loading_frames=True)
predictor.reset_state(inference_state)
print(inference_state["video_height"], inference_state["video_width"])

frame loading (JPEG):   4%|▎         | 13/360 [00:00<00:26, 12.94it/s]

133 53


frame loading (JPEG): 100%|██████████| 360/360 [00:11<00:00, 30.94it/s]


In [12]:
print(inference_state["video_height"], inference_state["video_width"])

133 53


In [15]:
def compute_image_dimensions(img_dir=FRAMES_PATH):
    widths = []
    heights = []
    
    # Iterate over files in the directory
    for filename in os.listdir(img_dir):
        if filename.lower().endswith(".jpg"):
            img_path = os.path.join(img_dir, filename)
            try:
                with Image.open(img_path) as img:
                    w, h = img.size
                    widths.append(w)
                    heights.append(h)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    
    if not widths or not heights:
        print("No images processed successfully.")
        return
    
    avg_width = np.mean(widths)
    avg_height = np.mean(heights)
    min_width = np.min(widths)
    max_width = np.max(widths)
    min_height = np.min(heights)
    max_height = np.max(heights)
    
    print("First width:", widths[0])
    print("First height:", heights[0])
    print("Average width:", avg_width)
    print("Average height:", avg_height)
    print("Min width:", min_width)
    print("Min height:", min_height)
    print("Max width:", max_width)
    print("Max height:", max_height)

In [16]:
compute_image_dimensions()

First width: 53
First height: 133
Average width: 49.28888888888889
Average height: 90.30555555555556
Min width: 18
Min height: 63
Max width: 87
Max height: 133


We observe that the "inference state" dictionary assigns its video_height and video_width according to the dimensions of the *first* frame - i.e. the model assumes all frames in the video are of the same dimension. So we need to pad all frames to the max height and width before feeding to the model.

In [1]:
def get_max_height_and_width(frames_path):
    widths = []
    heights = []
    
    # Iterate over files in the directory
    for filename in os.listdir(frames_path):
        if filename.lower().endswith(".jpg"):
            img_path = os.path.join(frames_path, filename)
            try:
                with Image.open(img_path) as img:
                    w, h = img.size
                    widths.append(w)
                    heights.append(h)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    
    if not widths or not heights:
        print("No images processed successfully.")
        return
    
    return np.max(heights), np.max(widths)

In [ ]:
def resize_and_pad_images(input_dir):

    output_dir = input_dir + "_resized"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    max_height, max_width = get_max_height_and_width(input_dir)
    pad_amounts = None
    
    for idx, filename in enumerate(os.listdir(input_dir)):
        if filename.lower().endswith(".jpg"):
            img_path = os.path.join(input_dir, filename)
            try:
                with Image.open(img_path) as img:
                    # Create a new image with the max dimensions and a black background
                    new_img = Image.new("RGB", (max_width, max_height), (0, 0, 0))
                    # Calculate padding amounts
                    pad_left = (max_width - img.width) // 2
                    pad_top = (max_height - img.height) // 2
                    # Paste the original image onto the center of the new image
                    new_img.paste(img, (pad_left, pad_top))
                    # Save the new image to the output directory
                    new_img.save(os.path.join(output_dir, filename))
                    
                    # Track pad amounts for the first image only
                    if idx == 0:
                        pad_amounts = (pad_left, pad_top)
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
    
    return pad_amounts

input_dir = FRAMES_PATH
output_dir = input_dir + "_resized"
first_bbox_pad = resize_and_pad_images(input_dir, output_dir)

In [8]:
print(f"Pad left: {first_bbox_pad[0]}, Pad top: {first_bbox_pad[1]}")

Pad left: 26, Pad top: 31


Did not work.